# 第1回：PathMNISTによる病理画像分類入門

## 1. 目的
- 医用画像データセット MedMNIST の扱いを学ぶ。
- 自作ライブラリ `exercise_logic` を活用し、効率的にAIを構築する。
- 畳み込みニューラルネットワーク（CNN）の学習プロセスを理解する。

In [ ]:
# 環境セットアップ
!pip install medmnist -q
import sys, os

# リポジトリをクローン
!rm -rf /tmp/MedMNIST-Exercise
!git clone https://github.com/kshimoji8/MedMNIST-Exercise-Public.git /tmp/MedMNIST-Exercise -q

sys.path.insert(0, '/tmp/MedMNIST-Exercise')

# モジュールをインポート
sys.modules.pop('exercise_logic', None)
import exercise_logic

exercise_logic.initialize_environment()

print("✓ セットアップが完了しました。")

# --- 2. データのロード ---
(x_train, y_train), (x_test, y_test), info = exercise_logic.load_and_preprocess('pathmnist')
print(f"✓ データロード完了: {x_train.shape}, Label: {y_train.shape}")

In [ ]:
# 2. シンプルなCNNモデルを構築
model = exercise_logic.build_model(
    input_shape=(28, 28, 3), 
    num_classes=len(info['label']), 
    model_type='simple'
)

model.summary()

In [ ]:
# 3. 学習の実行
history = model.fit(
    x_train, y_train, 
    epochs=10, 
    validation_split=0.1, 
    batch_size=128
)

In [ ]:
# 4. 評価と分析
exercise_logic.plot_history(history)
exercise_logic.show_evaluation_reports(model, x_test, y_test, info['label'])